In [1]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.initializers import RandomUniform
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
import numpy as np 
np.set_printoptions(threshold=np.inf)

# construct entity list:
MAX_NUM_ENTITIES = 256
EMBEDDED_ENTITY_LENGTH = 74
MISSING_BIAS = -1
MAX_COORD_VALUE_GAME_UNITS = 256
# NUM_SPATIAL_BUCKETS = 64

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes]).astype(int)

def prodToRadius(prod):
    return (prod+70)*12/85 

def encodeSpatialCoord(data):
    gameUnitCompressionFactor = 40
    dataToUintRange = 255 * (data.astype(int) / (2 * MAX_COORD_VALUE_GAME_UNITS) + 0.5)
    inRangeData = np.clip(dataToUintRange, 0, 255)
    return np.unpackbits(inRangeData.astype(np.uint8)).reshape(-1, 8)

def replaceNoneWith(array, replacement):
    return np.where(array == None, replacement, array)

FRIENDLY_ENCODING_POSITION = 32 + 2
ENEMY_ENCODING_POSITION = FRIENDLY_ENCODING_POSITION + 1

def getEncodedEmbedding(rawData, playerN):
    maxShips = 200 # sqrt(max)+1 = 15
    maxProduction = 100
    minProduction = 15
    maxRadius = 100 # for very spread out fleets

    unitType = get_one_hot((rawData[:, 0] == 'fleet').astype(int), 
                           2)
    # row 2 is player, row 3 enemy, row 4 neutral
    friendly = get_one_hot((rawData[:, 1].astype(int) != playerN).astype(int) + (rawData[:, 2]).astype(int), 
                           3)
    # TODO: this is a bit granular...
    ships = get_one_hot(np.sqrt(np.minimum((rawData[:, 3]).astype(int), maxShips)).astype(int),
                        int(np.sqrt(maxShips) + 1) )
    prodCleaned = replaceNoneWith(rawData[:, 4], 0).astype(int)
    production = get_one_hot(np.sqrt(np.minimum(prodCleaned, maxProduction)).astype(int),
                             int(np.sqrt(maxProduction) + 1) )
    radius = get_one_hot(np.sqrt(np.minimum(rawData[:, 5].astype(int), maxRadius)).astype(int),
                        int(np.sqrt(maxRadius) + 1) )
    x = encodeSpatialCoord(rawData[:, 6])
    y = encodeSpatialCoord(rawData[:, 7])
    
    targetIndices = rawData[:, 8]
    # 0-indexed, 3 players (incl. neutral)
    targetIndices[targetIndices != None] = targetIndices[targetIndices != None] - 4

    targetX = np.copy(targetIndices)
    # If fleet, then get X of target. Otherwise, get X of the planet.
    targetX[targetX != None] = rawData[targetX[targetX != None].astype(int), 6]
    targetX[targetX == None] = rawData[targetX == None, 6]
    targetX = encodeSpatialCoord(targetX)

    targetY = np.copy(targetIndices)
    # If fleet, then get Y of target. Otherwise, get Y of the planet.
    targetY[targetY != None] = rawData[targetY[targetY != None].astype(int), 7]
    targetY[targetY == None] = rawData[targetY == None, 7]
    targetY = encodeSpatialCoord(targetY)

    finalEmbedding = np.concatenate((x, y, targetX, targetY, unitType, friendly, ships, production, radius), axis=1)
    return finalEmbedding


# take preprocessed entities, then add -1e9 biases to any entries to create 512 length array
# TODO: They say their bias is -1e9????????
def getEntityEmbedding(frame, playerN):
     # 3 for users
    entities = [e for i, e in enumerate(frame['items']) if e['type'] != 'user' and i < MAX_NUM_ENTITIES + 3]
    rawData = np.array([[e.get(j) for j in ['type', 'owner', 'neutral', 's', 'p', 'r', 'x', 'y', 'target']] 
                          for e in entities])
    embedding = getEncodedEmbedding(rawData, int(playerN))
    amountToPad = MAX_NUM_ENTITIES - len(embedding)
    return np.pad(embedding, pad_width=((0,amountToPad), (0,0)), mode='constant', constant_values=MISSING_BIAS)

def getGameEntityEmbeddings(game):
    return np.array([getEntityEmbedding(frame, game['playerN']) for frame in game['frames']])



In [3]:
# import json 
# import numpy as np

# with open('D:\GalconZero\Waffle\\train_test_SMALL.jsonl') as f:
#     for line in f:
#         testGame = json.loads(line)
#         testEntityEmbeddings = getGameEntityEmbeddings(testGame)
#         testX = testEntityEmbeddings
#         testY = getTrainYForGame(testGame)

In [4]:
def rotateVec(angle, xs, ys):
    theSin = np.sin(np.deg2rad(angle))
    theCos = np.cos(np.deg2rad(angle))
    newXs = theCos * xs - theSin * ys
    newYs = theSin * xs + theCos * ys
    return newXs, newYs

# augment training data by reflecting about X axis
def augmentDataMirror(original):
    data = original.copy()
    for frame_i, frameEntities in enumerate(original):
        mask = np.sum(frameEntities[...,0:8], -1) >= 0
    
        newXs = 255 - np.packbits(frameEntities[mask,0:8], -1)
        data[frame_i,mask,0:8] = np.unpackbits(newXs, -1)
        
        newTargetXs = 255 - np.packbits(frameEntities[mask,16:24], -1)
        data[frame_i,mask,16:24] = np.unpackbits(newTargetXs, -1)

    return data
    
# augment training data by rotating about origin
def augmentDataRotate(original, angleDegrees):
    data = original.copy()
    for frame_i, frameEntities in enumerate(original):
        mask = np.sum(frameEntities[...,0:8], -1) >= 0
        
        entityXs = np.packbits(frameEntities[mask,0:8], -1) - 255/2
        entityYs = np.packbits(frameEntities[mask,8:16], -1) - 255/2
        newXs, newYs = rotateVec(angleDegrees, entityXs, entityYs) 
        newXs = np.round(newXs + 255/2).astype(np.uint8)
        newYs = np.round(newYs + 255/2).astype(np.uint8)
        data[frame_i,mask,0:8] = np.unpackbits(newXs, -1)
        data[frame_i,mask,8:16] = np.unpackbits(newYs, -1)

        targetXs = np.packbits(frameEntities[mask,16:24], -1) - 255/2
        targetYs = np.packbits(frameEntities[mask,24:32], -1) - 255/2
        newTargetXs, newTargetYs = rotateVec(angleDegrees, targetXs, targetYs) 
        newTargetXs = np.round(newTargetXs + 255/2).astype(np.uint8)
        newTargetYs = np.round(newTargetYs + 255/2).astype(np.uint8)
        data[frame_i,mask,16:24] = np.unpackbits(newTargetXs, -1)
        data[frame_i,mask,24:32] = np.unpackbits(newTargetYs, -1)

    return data
    
# Probably only useful for predicting win rate, not predicting the right move.
def augmentDataSwap(Xs, Ys):
    Xs = Xs.copy()
    Ys = Ys.copy()
    
    Xs[:,:,[FRIENDLY_ENCODING_POSITION,ENEMY_ENCODING_POSITION]] = Xs[:,:,[ENEMY_ENCODING_POSITION,FRIENDLY_ENCODING_POSITION]]
    Ys = 1 - Ys
    
    return Xs, Ys

def getFullAugmentedData(Xs, Ys):
    allXs = Xs.copy()
    allYs = Ys.copy()
    
    allXs = np.concatenate([allXs, augmentDataMirror(allXs)])
    allYs = np.concatenate([allYs, allYs])
    
    allXs = np.concatenate([allXs, augmentDataRotate(allXs, 180)])
    allYs = np.concatenate([allYs, allYs])
    
    allXs = np.concatenate([allXs, augmentDataRotate(allXs, 15), augmentDataRotate(allXs, -15)])
    allYs = np.concatenate([allYs, allYs, allYs])

    swappedXs, swappedYs = augmentDataSwap(allXs, allYs)
    allXs = np.concatenate([allXs, swappedXs])
    allYs = np.concatenate([allYs, swappedYs])
    
    return allXs, allYs

# TESTS:    
def testAugmentDataMirrorReversible():
    oldData = testEntityEmbeddings
    newData = augmentDataMirror(oldData)
    doubleReverse = augmentDataMirror(newData)
    assert(np.array_equal(oldData, doubleReverse))
    assert(not np.array_equal(oldData, newData))
    
def testAugmentDataRotateReversible_180():
    oldData = testEntityEmbeddings
    newData = augmentDataRotate(oldData, 180)
    doubleReverse = augmentDataRotate(newData, 180)
    assert(np.array_equal(oldData, doubleReverse))
    assert(not np.array_equal(oldData, newData))
    
def testAugmentDataRotateReversible_90(): 
    oldData = testEntityEmbeddings
    new1 = augmentDataRotate(oldData, 90)
    new2 = augmentDataRotate(new1, 90)
    new3 = augmentDataRotate(new2, 90)
    new4 = augmentDataRotate(new3, 90)
    assert(np.array_equal(oldData, new4))
    assert(not np.array_equal(oldData, new3))
    assert(not np.array_equal(oldData, new2))
    assert(not np.array_equal(oldData, new1))
    
def testAugmentDataRotateReversible_anyAngle():
    oldData = testEntityEmbeddings
    
    # to avoid rounding issues... same rounding issues will happen when rotated by 90 degrees, so they cancel!
    dataDirect = augmentDataRotate(oldData, 12.345)
    assert(not np.array_equal(dataDirect, oldData))
    
    dataIndirect = augmentDataRotate(oldData, 90)
    assert(not np.array_equal(dataIndirect, oldData))
    dataIndirect = augmentDataRotate(dataIndirect, 12.3452)
    assert(not np.array_equal(dataIndirect, oldData))
    dataIndirect = augmentDataRotate(dataIndirect, -90)
    assert(not np.array_equal(dataIndirect, oldData))

    assert(np.array_equal(dataDirect, dataIndirect))

def testAugmentDataSwapReversible():
    oldXs = np.copy(testX)
    oldYs = np.copy(testY)
    
    newX, newY = augmentDataSwap(oldXs, oldYs)
    restoredX, restoredY = augmentDataSwap(newX, newY)
    
    assert(not np.array_equal(oldXs, newX))
    assert(not np.array_equal(oldYs, newY))
    
    assert(np.array_equal(oldXs, restoredX))
    assert(np.array_equal(oldYs, restoredY))
        
# testAugmentDataMirrorReversible()
# testAugmentDataRotateReversible_180()
# testAugmentDataRotateReversible_90()
# testAugmentDataRotateReversible_anyAngle()
# testAugmentDataSwapReversible()
# x, y = getFullAugmentedData(testX, testY)
    
# augmentDataSwap(testEntityEmbeddings, Y)
    # TODO: should the data be rotated any more in any direction? maybe a few degrees in either direction etc?

In [ ]:
# from keras_transformer.position import TransformerCoordinateEmbedding
# from keras_transformer.transformer import TransformerACT, TransformerBlock

# inputLayer = Input(shape=testEmbedding.shape)

# # add_coordinate_embedding(inputLayer, step=step)

# # transformerDepth = 3

# # transformer_block = TransformerBlock(
# #     name='transformer',
# #     num_heads=2, # how many heads?
# #     residual_dropout=0.1, # Should I use dropout?
# #     attention_dropout=0.1,
# #     use_masking=True)
# # add_coordinate_embedding = TransformerCoordinateEmbedding(
# #     transformerDepth,
# #     name='coordinate_embedding')
    
# # output = inputLayer # shape: (<batch size>, <sequence length>, <input size>)
# # for step in range(transformerDepth):
# #     output = transformer_block(
# #         )

# model = Sequential()
# model.add(output)
# plot_model(model, to_file='models/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
NUM_SPATIAL_BUCKETS = 32
UINT8_TO_BUCKET = (NUM_SPATIAL_BUCKETS - 1)/255

# TODO: Implement this as a LAYER instead!!!
def scatterEntitiesIntoMapLayer(gameEntityEmbeddings):
    frameMapDataShape = (len(gameEntityEmbeddings), NUM_SPATIAL_BUCKETS, NUM_SPATIAL_BUCKETS, EMBEDDED_ENTITY_LENGTH - 16)
    gameMapData = np.zeros(frameMapDataShape)
    for i, frameEntities in enumerate(gameEntityEmbeddings):
        mask = np.sum(frameEntities[...,0:8], -1) >= 0
        entityDataOnly = frameEntities[mask]
        xIndex = (np.packbits(entityDataOnly[...,0:8], -1) * UINT8_TO_BUCKET).astype(int)
        yIndex = (np.packbits(entityDataOnly[...,8:16], -1) * UINT8_TO_BUCKET).astype(int)
        indices = np.column_stack((xIndex, yIndex))
        for e,[x,y] in enumerate(indices):
            gameMapData[i,x,y] += entityDataOnly[e,16:]
    
    # TODO: could decrease precision probably to speed things up?
    return gameMapData.astype(np.int32)
                
# testSpatialEmbedding = scatterEntitiesIntoMapLayer(testEntityEmbeddings)
# print(testSpatialEmbedding.dtype)

In [ ]:
import tensorflow as tf
from tensorflow import TensorShape

class ScatterLayer(Layer):
    
    def __init__(self, numBuckets):
        super(ScatterLayer, self).__init__()
        self.numBuckets = numBuckets
        self.uintToBucket = (numBuckets - 1)/255
        self.bitToIntMatrix = tf.constant([128, 64, 32, 16, 8, 4, 2, 1], dtype=tf.float32)
    
    def build(self, inputShape):
        self.outputShape = inputShape[:-2] + (self.numBuckets, self.numBuckets, inputShape[-1]) 
    
    def call(self, input):
        xIndices = tf.reduce_sum(input[...,0:8] * self.bitToIntMatrix, -1)
        xIndices = tf.math.scalar_mul(self.uintToBucket, xIndices)
        xIndices = tf.math.round(xIndices)
        presentMask = tf.math.greater_equal(xIndices, 0)
        
        # HACK: add 0s to (0,0) for a no-op
        xIndices = tf.clip_by_value(xIndices, 0, self.numBuckets - 1)
        
        yIndices = tf.reduce_sum(input[...,8:16] * self.bitToIntMatrix, -1)
        yIndices = tf.math.scalar_mul(self.uintToBucket, yIndices)
        yIndices = tf.math.round(yIndices)
        # HACK: add 0s to (0,0) for a no-op
        yIndices = tf.clip_by_value(yIndices, 0, self.numBuckets - 1)
        
        # missing entities will be less than 0 and masked out
        
        bitMask = tf.cast(presentMask, tf.float)
    
#         scatterIndicesX = tf.boolean_mask(xIndices, presentMask, 0)
#         scatterIndicesY = tf.boolean_mask(yIndices, presentMask, 0)
#         scatterIndices = tf.reshape(scatterIndices, [-1, presentMask.shape[1], *scatterIndices.shape[2:]])
        
#         updates = tf.boolean_mask(input, presentMask)
#         updates = tf.reshape(updates, [-1, presentMask.shape[1], 74])

        # boolean_mask not working. Trying slicing instead?
#         print(tf.math.count_nonzero(presentMask))

#         SCREW IT. JUST MULTIPLY STUFF BY 0????


#         
        
#         scattered = tf.scatter_nd(
#             scatterIndices, updates, shape=self.outputShape
#         )
#         print(input.shape)
#         print(presentMask.shape)
#         print(scatterIndicesX.shape)
#         print(scatterIndicesY.shape)
#         print("asdf")
#         print(updates.shape)
        return tf.where(presentMask, input * presentMask


# Spatial Encoder:
#     Inputs: map, entity_embeddings
#     Outputs:
#         embedded_spatial - A 1D tensor of the embedded map
#         map_skip - Tensors of the outputs of intermediate computations
def spatial_encoder_model(entityEmbeddings):
    inputShape = (MAX_NUM_ENTITIES, EMBEDDED_ENTITY_LENGTH)
    
    model = Sequential()
    model.add(InputLayer(input_shape=inputShape))
    model.add(ScatterLayer(numBuckets=NUM_SPATIAL_BUCKETS))
#     model.add(Conv2D(32,1,activation='relu'))
    
    return model

model = spatial_encoder_model(testEmbedding)
# model.summary()
model.predict(X[0:2],  steps=1)

In [ ]:
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock

def universal_transformer_alphaspark_model(
        inputShape: int, 
        transformer_depth: int,
        num_heads: int, 
        transformer_dropout: float = 0.1,
        confidence_penalty_weight: float = 0.1):
    
    # TODO: pass this in maybe?
    inputLayer = Input(shape=inputShape, name='inputLayerrrr') # TODO: can probably change dtype to binary or something?

    coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth,
        name='coordinate_embedding')
#     transformer_act_layer = TransformerACT(name='adaptive_computation_time')
    transformer_block = TransformerBlock(
        name='transformerASDF', 
        num_heads=num_heads,
        residual_dropout=transformer_dropout, #  Are these dropouts reasonable???
        attention_dropout=transformer_dropout,
        use_masking=True, 
        vanilla_wiring=False)
    output_softmax_layer = Softmax(name='predictionsIGuess')

#     next_step_input, embedding_matrix = embedding_layer(word_ids)
    next_step_input = inputLayer
#     act_output = next_step_input

    for i in range(transformer_depth):
        next_step_input = coordinate_embedding_layer(next_step_input, step=i)
        next_step_input = transformer_block(next_step_input)
#         next_step_input, act_output = transformer_act_layer(next_step_input)

#     transformer_act_layer.finalize()
#     next_step_input = act_output

    # TODO: this softmax may or may not work.
    predictions = output_softmax_layer(next_step_input)
    model = Model(inputs=[inputLayer], outputs=[predictions])
    # Penalty for confidence of the output distribution, as described in
    # "Regularizing Neural Networks by Penalizing Confident
    # Output Distributions" (https://arxiv.org/abs/1701.06548)
#     confidence_penalty = K.mean(
#         confidence_penalty_weight *
#         K.sum(word_predictions * K.log(word_predictions), axis=-1))
#     model.add_loss(confidence_penalty)
    return model

inputShape = (MAX_NUM_ENTITIES, EMBEDDED_ENTITY_LENGTH)
model = universal_transformer_alphaspark_model(inputShape, 1, 2)
# print(model)
model.summary()

In [ ]:
import keras.layers
import keras.regularizers


parameters = {
    "kernel_initializer": "he_normal"
}

BATCH_NORM_MOMENTUM = 0.99
BATCH_NORM_EPSILON = 0.001
L2_WEIGHT_DECAY = 0.001

def createResNetBlock(
    numFilters,
    stage=0,
    block=0,
    kernel_size=3
):
    def f(prev):
        y = prev
        # TODO: is axis=-1 default correct for batchnormalization?
        y = keras.layers.BatchNormalization(
                                        momentum=BATCH_NORM_MOMENTUM,
                                        epsilon=BATCH_NORM_EPSILON)(y)
        y = keras.layers.Activation("relu")(y)
        y = keras.layers.Conv2D(numFilters, kernel_size, padding="same", use_bias=False, 
                                kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY), **parameters)(y)

        y = keras.layers.BatchNormalization(
                                        momentum=BATCH_NORM_MOMENTUM,
                                        epsilon=BATCH_NORM_EPSILON)(y)
        y = keras.layers.Activation("relu")(y)
        y = keras.layers.Conv2D(numFilters, kernel_size, padding="same", use_bias=False, 
                                kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY), **parameters)(y)

        y = keras.layers.Add()([y, prev])

        return y
    return f

def createSpatialEncoder(prev):
    # initial kernel size 1 conv
    prev = Conv2D(64, 1, padding='same', kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(prev)
    prev = BatchNormalization()(prev)
    prev = Activation("relu")(prev)

    # downsampling
    prev = Conv2D(32, 4, strides=2, padding='same', kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(prev)
    prev = BatchNormalization()(prev)
    prev = Activation("relu")(prev)

    # TODO: ResBlocks

    prev = createResNetBlock(32)(prev)
    prev = createResNetBlock(32)(prev)
    prev = createResNetBlock(32)(prev)
    prev = createResNetBlock(32)(prev)
    
    prev = MaxPooling2D()(prev)
    prev = GlobalAveragePooling2D()(prev)
    prev = Dropout(0.1)(prev)
    prev = Dense(8, 
                 activation="relu",
                 kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY),
                 bias_regularizer=regularizers.l2(L2_WEIGHT_DECAY)
                )(prev)
    prev = Dropout(0.1)(prev)
    prev = Dense(1, 
                 activation="sigmoid",
                 kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY),
                 bias_regularizer=regularizers.l2(L2_WEIGHT_DECAY)
                )(prev)
    return prev


inputShape = (NUM_SPATIAL_BUCKETS, NUM_SPATIAL_BUCKETS, EMBEDDED_ENTITY_LENGTH - 16)
inputLayer = Input(shape=inputShape)
outputLayer = createSpatialEncoder(inputLayer)

model = Model(inputs=[inputLayer], outputs=[outputLayer])

model.summary()

# compiling the model
model.compile(
 optimizer = "adam", #just gonna hope this is correct
 loss = "binary_crossentropy", # IS THIS CORRECT ANYMORE??? I think this is appropriate because the model classifies win/loss with probability in range (0-1)
 metrics = ["accuracy"]
)


In [ ]:
plot_model(model, to_file='models/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.save_weights('models/model_resnet_2_pretrain.h5')

In [ ]:
model.load_weights('models/model_resnet_2_pretrain.h5')

In [ ]:
# def trainFromFile(model, filepath, numGamesPerEpisode=10):
#     games = []
#     with open(filepath) as f:
#         for line in f:
#             games.append(json.loads(line))
#             if len(games) >= numGamesPerEpisode:
#                 trainFromGames(model, games)
#                 games = []
#         if len(games) > 0:
#             trainFromGames(model, games)
#             games = []
       
    
# def trainFromGames(model, games):
#     print("Training on {} games".format(len(games)))
    
#     X = []
#     Y = []
     
#     for game in games:
#         X.extend(getGameEntityEmbeddings(game))
#         Y.extend(getTrainYForGame(game))

#     # TEMPORARY: Train on every 10th frame to avoid overfitting?
#     X = np.array(X)
#     Y = np.array(Y)
# #     X = np.array(X)[::10]
# #     Y = np.array(Y)[::10]    
    
#     model.fit(X, Y, validation_split=0.1, epochs=1, batch_size=32, shuffle=True)   

    
# trainFromFile(model, 'D:\GalconZero\Waffle\\train_test.jsonl', 1)

In [ ]:
# use only 1/SKIP_FRAME_RATIO frames to avoid overfitting
SKIP_FRAME_RATIO = 10

history = None

def trainFromFile(model, filepath, numGamesPerEpisode=10):
    games = []
    with open(filepath) as f:
        for line in f:
            games.append(json.loads(line))
            if len(games) >= numGamesPerEpisode:
                trainFromGames(model, games)
                return
                games = []
        if len(games) > 0:
            trainFromGames(model, games)
            games = []
            
def trainFromGames(model, games):
    print("Training on {} games".format(len(games)))
    
    X = []
    Y = []
    
    for game in games:
        game['frames'] = game['frames'][::SKIP_FRAME_RATIO]
        
        entityEmbeddings = getGameEntityEmbeddings(game)
        gameY = getTrainYForGame(game)
        
        augmentX, augmentY = getFullAugmentedData(entityEmbeddings, gameY)

        mapEmbedding = scatterEntitiesIntoMapLayer(augmentX)
        
        X.extend(mapEmbedding)
        Y.extend(augmentY)

    X = np.array(X)
    Y = np.array(Y)  
    
    global history
    newHistory = model.fit(X, Y, validation_split=0.2, epochs=1, batch_size=32, shuffle=True)   
#     newHistory = model.evaluate(X, Y, batch_size=32)
#     print(model.metrics_names)
#     print(newHistory)
    if history == None:
        history = newHistory
        print("none")
    for k in newHistory.history.keys():
        history.history[k].extend(newHistory.history[k])

    
# trainFromFile(model, 'D:\GalconZero\Waffle\\train_test_SMALL.jsonl', 1)

In [ ]:
import numpy as np
import json
import h5py

DATA_DIR = "D:/GalconZero/Waffle"
TRAINING_DATA_DB_FILENAME = "trainingData_db.hdf5"
SKIP_FRAME_RATIO = 10
NUM_GAMES_PER_CHUNK = 100
TEST_SPLIT_RATIO = 0.15
VALIDATION_SPLIT_RATIO = 0.15

X_TRAIN = "xTrain"
Y_TRAIN = "yTrain"
X_TEST = "xTest"
Y_TEST = "yTest"

def getTrainYForGame(game):
    playerWon = int(game["winnerN"] == game["playerN"])
    numFrames = len(game['frames'])
    return np.repeat(playerWon, numFrames)

def saveTrainingDataFromFile(directory, replayFilename):
    games = []
    with open(f"{directory}/{replayFilename}") as f:
        for line in f:
            games.append(json.loads(line))
            if len(games) >= NUM_GAMES_PER_CHUNK:
                saveTrainingData(games, directory)
                games = []
        if len(games) > 0:
            saveTrainingData(games, directory)
            games = []

def saveTrainingData(games, directory):
    print("Saving data for {} games".format(len(games)))
    
    X = []
    Y = []
    
    for game in games:
        # use only 1/SKIP_FRAME_RATIO frames to avoid overfitting
        game['frames'] = game['frames'][::SKIP_FRAME_RATIO]
        
        X.append(getGameEntityEmbeddings(game))
        Y.append(getTrainYForGame(game))

    X = np.concatenate(X)
    Y = np.concatenate(Y)
    assert(X.shape[0]== Y.shape[0])
    
    numTestSamples = round(X.shape[0] * TEST_SPLIT_RATIO)
    assert(numTestSamples > 0)
    
    xTrain = X[:-numTestSamples]
    xTest = X[-numTestSamples:]
    assert(xTrain.shape[0] + xTest.shape[0] == X.shape[0])
    
    yTrain = Y[:-numTestSamples]
    yTest = Y[-numTestSamples:]
    assert(yTrain.shape[0] + yTest.shape[0] == Y.shape[0])

    saveToHdf5(directory, xTrain, yTrain, xTest, yTest)
    
def saveToHdf5(directory, xTrain, yTrain, xTest, yTest):
    dbFilepath = f"{directory}/{TRAINING_DATA_DB_FILENAME}"
    with h5py.File(dbFilepath, 'a') as hf:
        appendData(hf, X_TRAIN, xTrain)
        appendData(hf, Y_TRAIN, yTrain)
        appendData(hf, X_TEST, xTest)
        appendData(hf, Y_TEST, yTest)
    
def appendData(hf, datasetName, data):
    if datasetName in hf:
        hf[datasetName].resize((hf[datasetName].shape[0] + data.shape[0]), axis = 0)
        hf[datasetName][-data.shape[0]:] = data
    else:
        addDataset(hf, datasetName, data)
        
# TODO: shuffle dataset
def addDataset(hf, datasetName, data):
    # TODO: can remove chunks=True? What is the default chunk size? is maxShape correct?
    maxShape = (None, *data.shape[1:])
    hf.create_dataset(datasetName, data=data, compression="gzip", chunks=True, maxshape=maxShape)
    
def fetchDataset(directory, hdf5Filename, xDatasetName, yDatasetName):
    dbFilepath = f"{directory}/{hdf5Filename}"
    with h5py.File(dbFilepath, 'r') as hf:
        # TODO: return a generator instead because this will exceed memory size soon, or maybe not necessary because I can index and it'll fetch lazily???
        # Or instead do a HDF5Matrix which might take care of this slicing stuff. Figure this out with print(len( etc))
        # "Not a dataset" error means we have closed the file.
#         xDataset = HDF5Matrix(dbFilepath, xDatasetName, end=100)
#         yDataset = HDF5Matrix(dbFilepath, yDatasetName, end=100)
        return hf[xDatasetName][0:2400], hf[yDatasetName][0:2400]
#     return xDataset, yDataset
     
def testFromHdf5File(model, directory, replayFilename):
    xTest, yTest = fetchDataset(directory, hdf5Filename, X_TEST, Y_TEST)
    
    augmentX, augmentY = getFullAugmentedData(xTest, yTest)
    
    # TODO: streamline this with a scatter layer instead of manual code.
    mapEmbedding = scatterEntitiesIntoMapLayer(augmentX)
    
    X = mapEmbedding
    Y = augmentY
    
    results = model.evaluate(X, Y, batch_size=32)
    print(results)
    
def trainFromHdf5File(model, directory, hdf5Filename):
    xTrain, yTrain = fetchDataset(directory, hdf5Filename, X_TRAIN, Y_TRAIN)
    
#     xTrain, yTrain = getFullAugmentedData(xTrain, yTrain)
    
    # TODO: streamline this with a scatter layer instead of manual code.
    xTrain = scatterEntitiesIntoMapLayer(xTrain)
    
    X = xTrain
    Y = yTrain
    
    history = model.fit(X, Y, validation_split=VALIDATION_SPLIT_RATIO, epochs=1, batch_size=32, shuffle=True)   
    
# # history = None
# a,b = fetchDataFromHdf5(DATA_DIR, "trainingData_db_small.hdf5", X_TRAIN, Y_TRAIN)
# print(a.shape, b.shape)


In [ ]:
import time

start = time.time()
trainFromFile(model, 'D:\GalconZero\Waffle\\train1.jsonl', 30)
print("TIME ELAPSED", time.time() - start)

start = time.time()
trainFromHdf5File(model, DATA_DIR, "trainingData_db_small.hdf5")
print("TIME ELAPSED", time.time() - start)

start = time.time()
trainFromHdf5File(model, DATA_DIR, "trainingData_db_small.hdf5")
print("TIME ELAPSED", time.time() - start)

start = time.time()
trainFromFile(model, 'D:\GalconZero\Waffle\\train1.jsonl', 30)
print("TIME ELAPSED", time.time() - start)

In [ ]:
saveTrainingDataFromFile("D:/GalconZero/Waffle", "train1.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train2.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train3.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train4.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train5.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train6.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train7.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train8.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train9.jsonl")
# saveTrainingDataFromFile("D:/GalconZero/Waffle", "train10.jsonl")

In [ ]:
# TEST TRAINING
# trainFromFile(model, 'D:\GalconZero\Waffle\\train_test_SMALL.jsonl', 1)
trainFromFile(model, 'D:\GalconZero\Waffle\\testData.jsonl', 30)

In [ ]:
def countLines(directory, filename):
    lines = 0
    with open(f"{directory}/{filename}") as f:
        for line in f:
            game = json.loads(line)
            lines += len(game["frames"])
    print("file ", lines)
    return lines

x = 0
# x += countLines("D:/GalconZero/Waffle", "train1.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train2.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train3.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train4.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train5.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train6.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train7.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train8.jsonl")
# x += countLines("D:/GalconZero/Waffle", "train9.jsonl")
print(x)
# should equal 1677147 / 0.085 = 19.7M
# 17,331,412

# 130110 for train1?

In [ ]:
history = None

In [ ]:
import matplotlib.pyplot as plt 
    
print(history.history.keys())  
   
plt.figure(1)  
   
# summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
# plt.plot(history.history['val_acc'])    
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
# summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
# plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  

In [ ]:
trainFromFile(model, 'D:\GalconZero\Waffle\\train6.jsonl', 10)

In [ ]:
# history = model.fit(X, Y, validation_split=0.33, epochs=1, batch_size=8, shuffle=True)   

In [ ]:
# Entity Encoder:




